# 01: Google Search Grounding with ADK

This notebook introduces Google Search grounding using the Agent Development Kit (ADK),
which provides explicit, traceable tool calls for web search.

## Learning Objectives

- Understand how Google Search grounding works with ADK
- Use the `KnowledgeGroundedAgent` to make grounded queries
- See explicit tool calls in the agent's reasoning
- Compare grounded vs non-grounded responses

In [ ]:
# Setup: Load environment and configure rich console
from aieng.agent_evals import (
    create_console,
    display_comparison,
    display_response,
    display_source_table,
)
from aieng.agent_evals.knowledge_qa import KnowledgeAgentConfig, KnowledgeGroundedAgent
from dotenv import load_dotenv
from google import genai
from rich.panel import Panel


console = create_console()
load_dotenv(verbose=True)

## 1. Understanding Google Search Grounding with ADK

The Agent Development Kit (ADK) provides a `GoogleSearchTool` that enables:

1. **Explicit Tool Calls**: The agent decides when to search and you can see each call
2. **ReAct Pattern**: Thought → Action → Observation loop is visible
3. **Traceable**: Every search query and result is logged
4. **Real-time Information**: Access to current web data

In [ ]:
# Initialize the knowledge agent (uses ADK with GoogleSearchTool internally)
agent = KnowledgeGroundedAgent()

console.print(
    Panel(
        f"[green]✓[/green] Knowledge Agent initialized\n[cyan]Model:[/cyan] {agent.model}",
        title="🔧 Setup Complete",
        border_style="green",
    )
)

## 2. Making Your First Grounded Query

In [ ]:
# Ask a question that requires current information
query = "What is the current population of Tokyo?"

console.print(f"\n[cyan]📝 Query:[/cyan] {query}\n")

console.print("[dim]Searching...[/dim]")
response = await agent.answer_async(query)

display_response(response, console=console, title="Tokyo Population")

# Show the tool calls made by the agent
if response.tool_calls:
    console.print("\n[bold cyan]🔧 Tool Calls Made:[/bold cyan]")
    for tc in response.tool_calls:
        console.print(f"  • {tc.get('name', 'unknown')}: {tc.get('args', {})}")

In [ ]:
# Display sources in a detailed table format
display_source_table(response, console=console)

## 3. Comparing Grounded vs Non-Grounded Responses

This is where grounding truly shines. We'll ask about Toronto's record single-day snowfall.

**Why this example works:**
- The record was set on **January 25, 2026** - after the model's training data cutoff
- Without grounding, the model can only guess based on historical data it was trained on
- With grounding, the model searches the web and finds the recent news about this event

This clearly demonstrates that grounding enables access to information the model couldn't possibly know from training alone.

In [ ]:
config = KnowledgeAgentConfig()
client = genai.Client(api_key=config.gemini_api_key)

# This question requires very recent information (Jan 2026)
# The non-grounded model will fail since its training data doesn't include this event
question = "Which day had the highest recorded snowfall in a single day in Toronto?"
expected_answer = "January 25, 2026"

console.print(f"\n[bold]Question:[/bold] {question}")
console.print(f"[dim]Expected Answer: {expected_answer}[/dim]\n")

# Without grounding - model relies on training data (cutoff before Jan 2026)
console.print("[dim]Generating without grounding...[/dim]")
response_no_grounding = client.models.generate_content(
    model=config.default_worker_model,
    contents=question,
)

# With grounding - agent uses Google Search tool
console.print("[dim]Generating with grounding (ADK agent)...[/dim]")
response_grounded = await agent.answer_async(question)

# Side-by-side comparison using our display utility
display_comparison(response_no_grounding.text, response_grounded, console=console)

# Show tool calls from the grounded response
if response_grounded.tool_calls:
    console.print("\n[bold cyan]🔧 Tool Calls (Grounded):[/bold cyan]")
    for tc in response_grounded.tool_calls:
        console.print(f"  • {tc.get('name', 'unknown')}: {tc.get('args', {})}")

# Check if the grounded response contains the correct answer
if expected_answer.lower() in response_grounded.text.lower() or "january 25" in response_grounded.text.lower():
    console.print("\n[green]✓ Grounded response contains the correct answer![/green]")
else:
    console.print("\n[yellow]⚠ Check the grounded response for accuracy[/yellow]")

## 4. Exercise: Try Your Own Queries

Try asking questions that:
- Require recent information (news, events, statistics)
- Need multiple facts combined
- Are about specific domains (sports, science, politics)

In [ ]:
# Try your own query
my_query = "What are the latest developments in fusion energy?"

console.print(f"[bold cyan]🔍 Query:[/bold cyan] {my_query}\n")

console.print("[dim]Searching the web...[/dim]")
my_response = await agent.answer_async(my_query)

display_response(my_response, console=console, title="Fusion Energy Developments")

# Show the tool calls
if my_response.tool_calls:
    console.print("\n[bold cyan]🔧 Tool Calls:[/bold cyan]")
    for tc in my_response.tool_calls:
        console.print(f"  • {tc.get('name', 'unknown')}: {tc.get('args', {})}")

## Summary

In this notebook, you learned:

1. How Google Search grounding works with ADK's `GoogleSearchTool`
2. How to use the `KnowledgeGroundedAgent` for grounded queries
3. How to see explicit tool calls in the agent's response
4. The difference between grounded and non-grounded responses

**Next**: In the next notebook, we'll explore the agent's system instructions and the evaluation dataset.

In [ ]:
console.print(
    Panel(
        "[green]✓[/green] Notebook complete!\n\n"
        "[cyan]Next:[/cyan] Open [bold]02_agent_basics.ipynb[/bold] to learn about the Knowledge Agent.",
        title="🎉 Done",
        border_style="green",
    )
)